In [ ]:
%%writefile recommendation.py

import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recommend(movie_name):
    movies_data = pd.read_csv('movies.csv')
    selected_features = ['genres','keywords','tagline','cast','director']
    for feature in selected_features:
      movies_data[feature] = movies_data[feature].fillna('')
    combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']
    vectorizer = TfidfVectorizer()
    feature_vectors = vectorizer.fit_transform(combined_features)
    similarity = cosine_similarity(feature_vectors)

    list_of_all_titles = movies_data['title'].tolist()
    find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
    close_match = find_close_match[0]
    index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

    ans=""
    i = 1
    for movie in sorted_similar_movies:
      index = movie[0]
      title_from_index = movies_data[movies_data.index==index]['title'].values[0]
      if (i<=12):
        ans=ans+(str(i)+'. '+title_from_index+'\n')
        i+=1

    return ans.strip('\n')

In [ ]:
!pip install pyngrok flask

In [ ]:
from pyngrok import ngrok
from flask import Flask, request
from recommendation import recommend

In [ ]:
port_no = 5000
app = Flask(__name__)
ngrok.set_auth_token("ngrok_token")
public_url = "ngrok_url"

@app.route("/generate", methods=["POST"])
def generate():
  movie_name = request.json["m_name"]
  output = recommend(movie_name)

  return {"output": output}, 201

ngrok_tunnel = ngrok.connect(port_no, bind_tls=True, hostname="ngrok_url")
if __name__ == '__main__':
  print(f"Ngrok tunnel: {ngrok_tunnel.public_url}")
  app.run(host="0.0.0.0", debug=True, port=port_no, use_reloader=False)